# To Do
1. Only works for single SRR files right now. Need to work through the logic statement maze and fix things for multiple SRR files to a single Sample
2. Quality checks

We're using os and glob to do bash script stuff.

In [1]:
import os
import glob
import pandas as pd
import numpy as np

#stores the working directory as a string
path = os.getcwd()

#ref folder name, ref genomes go here
ref_folder = 'references'

#data folder name, samples and runs go here
data = 'data'

#csv output folder name
csv = 'Ex'


# Get List of SRR#s to Automate Downloading Data

Use Georgia's metadata table to download sample runs. 

Metadata Table -> dataframe -> list of SRR#s (or SRR#s paired to biosample/experiment)

List of SRR#s -> SRA Toolkit -> fastq files -> salmon -> quant files

In [5]:
#read in SRR#s from metadata table provided by Georgia
sra_run_table = pd.read_csv(r'SraRunTable.csv', sep=",", header=0, index_col=0)
#print(sra_run_table['Run'][3:7])

#create a dict key of biosamples : srr numbers(could do SRX:SRR instead)
bio_sample_dic = sra_run_table.groupby('BioSample').agg({'Run':lambda x:x.tolist()})['Run'].to_dict()

# Building Index
Use salmon to build index. Download genomes from ensemble and store in refences folder. Will align to references in parallel. This is written to align to as many reference genomes/transcriptomes as are put in the references folder. 

In [6]:
#grabs stuff and returns a list of them-used throughout
def grab_ref(ref_folder1):
    files=glob.glob(ref_folder1+'/*')
    #print(files)
    return(files)

#builds Salmon index from given file names, puts indexes into folder, INDEX_references 
def salmon_index(genome):
    #os.system('salmon index -t '+str(genome)+' -i INDEX_'+str(genome))
    #print('salmon '+ genome)
    return 

#call grab_ref() and salmon_index() to build index for each reference genome
for i in grab_ref(ref_folder):
    salmon_index(i)

# Get Quants for Each Reference Genome on a Per Sample Basis
We want all of the runs from each sample to be fed into salmon at one time. This is currently robust to number of runs and paired vs unpaired, salmon determines library type automatically. Salmon says run files must be of same library type and aligned to same transcriptome. 

loops through indexes and samples (SRR#, biosample, experiment? - need to sort this out) and runs quant for each sample and each index. 

fastqdump downloads data on the fly so we are not running prefetch

still need to test this robustly 

In [20]:
index_names =[] #will use this later

for y in glob.glob('INDEX_'+ref_folder+'/*'):
    index_name = y.replace('INDEX_references/','').replace('.ffn.gz','')
    print('Index: '+index_name)
    index_names.append(index_name)
    #you could fairly easily change this to loop by experiment 
    #number or biosample number...need to talk about which one to choose
    for w in list(bio_sample_dic.keys())[601:602]:
        for q in bio_sample_dic[w]:
            dump_directory = 'data/'+w+'/'+q
            #os.system('mkdir data/'+w+'/'+q)
            #print('Dump Directory: '+dump_directory)
            #os.system('fastq-dump --outdir '+dump_directory+' -v --gzip --skip-technical --readids --split-3 --clip '+q)
        run_folders = grab_ref('data/'+w)
        output_name='quants/'+index_name+'/'+w+'_quant'
        print('Quant output:'+output_name)
        if len(run_folders)<1:
            print('Oops, no runs for '+w)
        if len(run_folders)==1:
            if len(grab_ref(run_folders[0]))<1:
                print('Oops, no runs for '+w)
            elif len(grab_ref(run_folders[0]))==1:
                print('One run for ')
                print('Salmon input: '+grab_ref(run_folders[0])[0])
                #os.system('salmon quant -i '+y+' -l A '+grab_ref(run_folders[0])[0]+' -p 8 --validateMappings -o '+output_name)
            elif len(grab_ref(run_folders[0]))>1:
                runs = grab_ref(run_folders[0])
                print('Multiple runs for ')
                if "_1" in runs[0] or "_2" in runs[0]:
                    print('Multiple paired runs for '+w)
                    input_name='-1'
                    for z in np.arange(0, len(runs)):
                        if "_1" in runs[z]:
                            input_name+= ' '+runs[z]
                    input_name+=' -2'
                    for z in np.arange(0, len(runs)):
                        if "_2" in runs[z]:
                            input_name+= ' '+runs[z]
                else:
                    print('Multiple runs for '+z)
                    input_name='-r'
                    for z in np.arange(0, len(runs)):
                        input_name+= ' '+runs[z]
                print('Salmon Input: '+input_name)
        if len(run_folders)>1:
            print('Multiple Folder Case')
            first_read_str = ''
            second_read_str = ''
            orphan_read = '-r'
            for run_folder in run_folders:
                if len(grab_ref(run_folder))<1:
                    print('Opps, no runs for '+run_folder)
                elif len(grab_ref(run_folder))==1:
                    print('One run for '+run_folder)
                    print('Salmon input: '+grab_ref(run_folder)[0])
                    orphan_read += ' '+grab_ref(run_folder)[0]
                    #os.system('salmon quant -i '+y+' -l A '+grab_ref(run_folders[0])[0]+' -p 8 --validateMappings -o '+output_name)
                elif len(grab_ref(run_folder))>1:
                    print('Multiple runs for '+run_folder)
                    runs=grab_ref(run_folder)
                    if "_1" in runs[0] or "_2" in runs[0]:
                        print('Multiple paired runs for '+w)
                        for z in np.arange(0, len(runs)):
                            if "_1" in runs[z]:
                                first_read_str += ' '+runs[z]
                        for z in np.arange(0, len(runs)):
                            if "_2" in runs[z]:
                                second_read_str+= ' '+runs[z]
                    else:
                        print('Multiple runs for '+run_folder)
                        for z in np.arange(0, len(runs)):
                            orphan_read+=runs[z]
            if len(orphan_read) > 2: 
                if len(first_read_str)>1 or len(second_read_str)>1:
                    input_name = orphan_read+first_read_str+second_read_Str
                else: 
                    input_name = orphan_read
            elif len(orphan_read) == 2:
                input_name = '-1'+first_read_str+' -2'+second_read_str
            print('Salmon Input: '+input_name)
            os.system('salmon quant -i '+y+' -l A '+input_name+' -p 8 --validateMappings -o '+output_name)
            #os.system('rm -r data/'+w) 

Index: Pseudomonas_aeruginosa_UCBPP-PA14_109
Quant output:quants/Pseudomonas_aeruginosa_UCBPP-PA14_109/SAMN02691047_quant
Multiple Folder Case
Multiple runs for data/SAMN02691047/SRR2099907
Multiple paired runs for SAMN02691047
Multiple runs for data/SAMN02691047/SRR2098932
Multiple paired runs for SAMN02691047
Multiple runs for data/SAMN02691047/SRR2098784
Multiple paired runs for SAMN02691047
Salmon Input: -1 data/SAMN02691047/SRR2099907/SRR2099907_1.fastq.gz data/SAMN02691047/SRR2098932/SRR2098932_1.fastq.gz data/SAMN02691047/SRR2098784/SRR2098784_1.fastq.gz -2 data/SAMN02691047/SRR2099907/SRR2099907_2.fastq.gz data/SAMN02691047/SRR2098932/SRR2098932_2.fastq.gz data/SAMN02691047/SRR2098784/SRR2098784_2.fastq.gz
Index: Pseudomonas_aeruginosa_PAO1_107
Quant output:quants/Pseudomonas_aeruginosa_PAO1_107/SAMN02691047_quant
Multiple Folder Case
Multiple runs for data/SAMN02691047/SRR2099907
Multiple paired runs for SAMN02691047
Multiple runs for data/SAMN02691047/SRR2098932
Multiple pair

# Storing Output

Copied Alex's code. 

Similar to running the quants, for each reference genome, we're going to grab the quants on a sample by sample basis and construct a data frame for each reference genome. The dataframes will be stored in a list. 

Not sure if row names are robust. I think they are, but I have not thought hard about it. 

In [5]:
#loop through indexes and build dataframe of output for each one
df_list = []
for i in index_names:
    glob_list = []
    replace = 'quants/'+i+'/'
    #print(i)
    for z in grab_ref('quants/'+i):
        #print(z)
        for file in glob.glob(z+'/*.sf'):
            glob_list.append(file)
            #print(file)
    expression_df = pd.DataFrame(
    pd.read_csv(file, sep="\t", index_col=0)["TPM"].
        rename(file.replace(replace, '').replace('_quant/quant.sf',''))
    for file in glob_list)
    df_list.append(expression_df)
    expression_df.to_csv(csv+'/aligned_to_'+i, sep='\t')
print(df_list)

[Name                    PGD1650835  PGD1650837  PGD1650839  PGD1650841  \
GD9-45172138_pstB_rep2   81.384071  138.613259   54.629463   60.874352   
GD1-45169164-WT_rep1    127.575897  219.302219   91.260433   91.858371   
GD3-45171150-pstB_rep1   91.179014  135.601506   63.020443   68.330303   
GD7-45169169_WT_rep2    116.865091  193.588654   74.004411   76.063677   

Name                    PGD1650843  PGD1650845  PGD1650847  PGD1650849  \
GD9-45172138_pstB_rep2   16.263993   20.311225  291.508020   16.548822   
GD1-45169164-WT_rep1     34.162917   38.739507  330.823388   27.026849   
GD3-45171150-pstB_rep1   19.360923   33.289960  239.189432   18.592331   
GD7-45169169_WT_rep2     28.681000   27.889116  260.181345   25.425216   

Name                    PGD1650851  PGD1650853  ...  PGD1662756  PGD1662758  \
GD9-45172138_pstB_rep2   24.734659   16.689345  ...   59.918662   70.914933   
GD1-45169164-WT_rep1     37.421190   28.794584  ...  112.652261  135.126782   
GD3-45171150-pstB_re